## 1. Import Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sklearn_crfsuite
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
print("Libraries imported successfully!")

## 2. Data Ingestion and Preparation

In [ ]:
# Read JSON data
with open('ingredient_and_quantity.json', 'r') as file:
    data = json.load(file)

# Preview data
print(f"Total number of recipes: {len(data)}")
print("Sample recipe:", data[0])

# Data quality checks
missing_counts = sum([1 for item in data if item is None or item == {}])
print(f"Recipes with missing data: {missing_counts}")

# Interpret fields (add your interpretations here)

## 3. Train-Validation Split

In [ ]:
train_data, val_data = train_test_split(data, test_size=0.3, random_state=42)
print(f"Training recipes: {len(train_data)}")
print(f"Validation recipes: {len(val_data)}")

## 4. Exploratory Data Analysis on Training Data

In [ ]:
# Example: Flatten all tokens into a list
all_tokens = []
for recipe in train_data:
    for ingredient in recipe.get("ingredients", []):
        all_tokens.extend(ingredient.split())

# Token frequency analysis
token_counts = Counter(all_tokens)
most_common_tokens = token_counts.most_common(10)
print("Most common tokens:", most_common_tokens)

# Plot top 10 tokens
tokens, counts = zip(*most_common_tokens)
plt.figure(figsize=(8,4))
sns.barplot(x=list(tokens), y=list(counts))
plt.title("Top 10 Most Frequent Tokens in Training Data")
plt.ylabel("Frequency")
plt.xticks(rotation=45)
plt.show()

## 5. Exploratory Data Analysis on Validation Data

In [ ]:
# Optional: Repeat similar analysis for validation data if desired

## 6. Feature Extraction for CRF Model

In [ ]:
def word2features(sent, i):
    word = sent[i]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        word1 = sent[i-1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent_labels):
    return sent_labels

def sent2tokens(sent):
    return sent

# Example placeholder: convert your train/val data to X_train, y_train
# X_train = [sent2features(s) for s in train_sents]
# y_train = [sent2labels(s) for s in train_labels]

## 7. Model Building and Training

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

# Example: crf.fit(X_train, y_train)
print("CRF model initialized. Train with crf.fit(X_train, y_train)")

## 8. Prediction and Model Evaluation

In [ ]:
# Example: y_pred = crf.predict(X_val)
# print(classification_report(y_val, y_pred))

# Example: plot confusion matrix
# labels = list(crf.classes_)
# cm = confusion_matrix(y_val_flat, y_pred_flat, labels=labels)
# plt.figure(figsize=(10,7))
# sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels)
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.title('Confusion Matrix')
# plt.show()

## 9. Error Analysis on Validation Data

In [ ]:
# Example template for error analysis
# errors = []
# for true_seq, pred_seq in zip(y_val, y_pred):
#     for true_label, pred_label in zip(true_seq, pred_seq):
#         if true_label != pred_label:
#             errors.append((true_label, pred_label))

# error_counts = Counter(errors)
# print("Most common errors:")
# print(error_counts.most_common(10))

## 10. Conclusion

In [ ]:
# Summarize key findings and suggest possible next steps.